In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

In [45]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        diff_low   =   (ohlcv_latest["open"][0] - ohlcv_latest["low"][0])
        diff_high  = - (ohlcv_latest["open"][0] - ohlcv_latest["high"][0])  

        detect_short  = range_width * self.K < diff_low
        detect_long = range_width * self.K < diff_high
        
        if detect_short and detect_long:
            print("detect both side")
            if diff_low < diff_high:
                detect_long = True
                detect_short = False
            else:
                detect_long = False
                detect_short = True

        print(range_width*self.K, diff_low, diff_high, detect_long, detect_short)

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.place_order("Buy", session.best_buy_price, 0.01, 600, "Open Long")    
                else:

                    session.place_order("Buy", session.best_buy_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.place_order("Sell", session.best_sell_price, 0.01, 600, "Open Short") 
                else:
                    session.place_order("Sell", session.best_sell_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [46]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%
back_tester.maker_fee_rate

0.001

In [50]:


print("start")
r = back_tester.run(Agent(), rbot.DAYS_BEFORE(10), 0)
#r = back_tester.run(Agent(), 0, 0)
print("end")




start
130.59999999999943 127.56999999999971 216.72999999999956 True False
214.2751999999996 194.09000000000015 65.06999999999971 False False
262.89439999999945 58.11999999999898 22.419999999998254 False False
266.96479999999923 68.33000000000175 7.379999999997381 False False
265.42239999999936 106.88000000000102 14.2599999999984 False False
280.73279999999914 3.2400000000016007 61.159999999999854 False False
191.95199999999897 108.15999999999985 70.84000000000015 False False
166.52319999999892 68.36999999999898 15.409999999999854 False False
167.62239999999935 21.760000000002037 42.38999999999942 False False
163.95679999999936 27.860000000000582 37.06000000000131 False False
146.73440000000002 19.340000000000146 32.86999999999898 False False
142.76800000000046 16.169999999998254 82.40000000000146 False False
117.19680000000051 213.3199999999997 107.39000000000306 False True
193.12800000000107 33.9800000000032 99.03999999999724 False False
213.87360000000047 35.52000000000044 34.8400000

In [51]:
import rbot
df = rbot.result_to_df(r)

In [52]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-01 14:00:00+00:00,2022-12-01 14:00:00.159000+00:00,0000-0001,0,Buy,True,Open,17136.27,0.01,171.3627,0.0,...,17136.27,0.01,171.3627,0.0,0.171363,-0.171363,0.01,Open Long,-0.171363,0.01
2022-12-02 14:00:00+00:00,2022-12-02 14:10:00.053000+00:00,0000-0002,0,Sell,True,Expire,0.00,0.00,0.0000,0.0,...,16997.89,0.02,339.9578,0.0,0.000000,0.000000,0.00,Doten Short,-0.171363,0.01
2022-12-04 00:00:00+00:00,2022-12-04 00:10:00.020000+00:00,0000-0003,0,Sell,True,Expire,0.00,0.00,0.0000,0.0,...,16949.21,0.02,338.9842,0.0,0.000000,0.000000,0.00,Doten Short,-0.171363,0.01
2022-12-05 16:00:00+00:00,2022-12-05 16:10:00.066000+00:00,0000-0004,0,Sell,True,Expire,0.00,0.00,0.0000,0.0,...,17234.33,0.02,344.6866,0.0,0.000000,0.000000,0.00,Doten Short,-0.171363,0.01
2022-12-07 08:00:00+00:00,2022-12-07 08:10:00.030000+00:00,0000-0005,0,Sell,True,Expire,0.00,0.00,0.0000,0.0,...,17005.57,0.02,340.1114,0.0,0.000000,0.000000,0.00,Doten Short,-0.171363,0.01
2022-12-09 14:00:00+00:00,2022-12-09 14:10:00.314000+00:00,0000-0006,0,Sell,True,Expire,0.00,0.00,0.0000,0.0,...,17242.62,0.02,344.8524,0.0,0.000000,0.000000,0.00,Doten Short,-0.171363,0.01


In [55]:
market = Market.open("BN", "BTCBUSD")
ohlcv = market.ohlcv(rbot.NOW()-rbot.DAYS(20), 0, 60*60)


In [56]:
chart = Chart(800, 400, ohlcv)
chart.show()

Loading BokehJS ...

main


In [58]:
from bokeh.models.glyphs import Step
from bokeh.models import ColumnDataSource

chart2 = Chart(800, 400, ohlcv)
p = chart2.new_figure('profit', 100, 'profit')

p = chart2.get_figure('profit')

p.step(x=df['update_time'], y=df['sum_profit'])

p = chart2.get_figure('main')

buy_df = df[df['order_side'] == "Buy"]
p.scatter(x=buy_df['update_time'], y=buy_df['order_price'], marker='triangle', size=10, color="#6699ff")


sell_df = df[df['order_side'] == "Sell"]
p.scatter(x=sell_df.index, y=sell_df['order_price'], marker='inverted_triangle', size=10, color="#ff8080")
#p.scatter(x=df.index, y=df['order_price'], marker='triangle', size=14)


chart2.show()

Loading BokehJS ...

main
profit


In [15]:
buy_df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-02 14:00:00+00:00,2022-12-02 14:00:00.070000+00:00,0000-0002,0,Buy,True,Open,16997.89,0.01,169.9789,0.00,...,16997.89,0.01,169.9789,0.0000,0.169979,-0.169979,0.01,Open Long,-0.169979,0.01
2022-12-05 16:00:00+00:00,2022-12-05 16:00:00.034000+00:00,0000-0004,0,Buy,True,Close,16888.07,0.01,168.8807,17234.33,...,17234.33,0.01,172.3433,-3.4626,0.172343,-3.634943,0.01,doten Long,-5.240884,0.00
2022-12-05 16:00:00+00:00,2022-12-05 16:00:00.034000+00:00,0000-0004,1,Buy,True,Open,17234.33,0.01,172.3433,0.00,...,17234.33,0.01,172.3433,0.0000,0.172343,-0.172343,0.01,doten Long,-5.413227,0.01
2022-12-09 14:00:00+00:00,2022-12-09 14:00:00.035000+00:00,0000-0006,0,Buy,True,Close,16967.98,0.01,169.6798,17242.86,...,17242.86,0.01,172.4286,-2.7488,0.172429,-2.921229,0.01,doten Long,-11.337315,0.00
2022-12-09 14:00:00+00:00,2022-12-09 14:00:00.035000+00:00,0000-0006,1,Buy,True,Open,17242.86,0.01,172.4286,0.00,...,17242.86,0.01,172.4286,0.0000,0.172429,-0.172429,0.01,doten Long,-11.509744,0.01


In [16]:
sell_df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-01 14:00:00+00:00,2022-12-01 14:10:00.127000+00:00,0000-0001,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17135.25,0.01,171.3525,0.0000,0.000000,0.000000,0.00,Open Short,0.000000,0.00
2022-12-04 02:00:00+00:00,2022-12-04 02:00:00.601000+00:00,0000-0003,0,Sell,True,Close,16997.89,0.01,169.9789,16888.07,...,16888.07,0.01,168.8807,-1.0982,0.168881,-1.267081,-0.01,Doten Short,-1.437060,0.00
2022-12-04 02:00:00+00:00,2022-12-04 02:00:00.601000+00:00,0000-0003,1,Sell,True,Open,16888.07,0.01,168.8807,0.00,...,16888.07,0.01,168.8807,0.0000,0.168881,-0.168881,-0.01,Doten Short,-1.605940,-0.01
2022-12-08 20:00:00+00:00,2022-12-08 20:00:00.277000+00:00,0000-0005,0,Sell,True,Close,17234.33,0.01,172.3433,16967.98,...,16967.98,0.01,169.6798,-2.6635,0.169680,-2.833180,-0.01,Doten Short,-8.246407,0.00
2022-12-08 20:00:00+00:00,2022-12-08 20:00:00.277000+00:00,0000-0005,1,Sell,True,Open,16967.98,0.01,169.6798,0.00,...,16967.98,0.01,169.6798,0.0000,0.169680,-0.169680,-0.01,Doten Short,-8.416087,-0.01


In [ ]:
rbot.init_debug_log()
bn.download(10)

2022-12-11T05:24:53.655Z DEBUG [rbot::exchange::binance] download after 1670371199890000 1670563493654579
2022-12-11T05:24:53.655Z DEBUG [rbot::db::sqlite] chunk: 2022-12-06T23:59:59.890000 -> 2022-12-09T05:24:53.654579
2022-12-11T05:24:53.655Z DEBUG [rbot::db::sqlite] DAY: 2022-12-06T00:00:00.000000
2022-12-11T05:24:53.655Z DEBUG [rbot::db::sqlite] DAY: 2022-12-07T00:00:00.000000
2022-12-11T05:24:53.655Z DEBUG [rbot::db::sqlite] DAY: 2022-12-08T00:00:00.000000
2022-12-11T05:24:53.655Z DEBUG [rbot::db::sqlite] DAY: 2022-12-09T00:00:00.000000
2022-12-11T05:24:53.655Z DEBUG [rbot::exchange::binance] GAP TIME: [TimeChunk { start: 1670371199890000, end: 1670563493654579 }]
2022-12-11T05:24:53.655Z DEBUG [rbot::exchange::binance] GAP DAYS: [1670284800000000, 1670371200000000, 1670457600000000, 1670544000000000]
2022-12-11T05:24:53.655Z DEBUG [rbot::exchange::binance] download url = https://data.binance.vision/data/spot/daily/trades/BTCBUSD/BTCBUSD-trades-2022-12-06.zip
2022-12-11T05:24:53.6

9707483

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time

In [ ]:
df['order_side'] == rbot.OrderSide.Buy

create_time
2022-12-01 14:00:00+00:00    False
2022-12-02 14:00:00+00:00     True
2022-12-04 02:00:00+00:00    False
2022-12-04 02:00:00+00:00    False
2022-12-05 16:00:00+00:00     True
2022-12-05 16:00:00+00:00     True
Name: order_side, dtype: bool